In [20]:
import re
import nltk
import string
import math
import csv
import numpy as np
#import feature_matrix
#import tensorflow
from nltk.corpus import wordnet
import RandomTree as rt
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import one_hot
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

/home/rashi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/rashi/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [21]:
def cap(sentence):
    uppers = [l for l in sentence if l.isupper()]
    words = len(sentence.split())
    return (len(uppers)/words)

In [22]:
def punc(sentence):
    count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
    #a_chars =  count(a, string.ascii_letters)
    a_punct = count(a, string.punctuation)
    #print(a_chars)
    words = len(sentence.split())
    return (a_punct/words)

In [23]:
def chars(sentence):
    count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
    a_chars =  count(a, string.ascii_letters)
    words = len(sentence.split())
    return (a_chars/words)

In [24]:
def lesk(context_sentence, ambiguous_word, pos=None, synsets=None):
    """Return a synset for an ambiguous word in a context.

    :param iter context_sentence: The context sentence where the ambiguous word
    occurs, passed as an iterable of words.
    :param str ambiguous_word: The ambiguous word that requires WSD.
    :param str pos: A specified Part-of-Speech (POS).
    :param iter synsets: Possible synsets of the ambiguous word.
    :return: ``lesk_sense`` The Synset() object with the highest signature overlaps.

    This function is an implementation of the original Lesk algorithm (1986) [1].

    Usage example::

        >>> lesk(['I', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'money', '.'], 'bank', 'n')
        Synset('savings_bank.n.02')

    [1] Lesk, Michael. "Automatic sense disambiguation using machine
    readable dictionaries: how to tell a pine cone from an ice cream
    cone." Proceedings of the 5th Annual International Conference on
    Systems Documentation. ACM, 1986.
    http://dl.acm.org/citation.cfm?id=318728
    """

    context = set(context_sentence)
    if synsets is None:
        synsets = wordnet.synsets(ambiguous_word)
    #print("\n\n" + " :::   " + pos)
    #print(synsets)
           
    if pos=='JJ' or pos=='JJR' or pos=='JJS' :
        synsets = [ss for ss in synsets if (str(ss.pos()) == 'a' or str(ss.pos()) == 's')]
    if pos=='RB' or pos=='RBR' or pos=='RBS' :
        synsets = [ss for ss in synsets if str(ss.pos()) == 'r']
    #print("End")
    
    if not synsets:
        return None

    _, sense = max(
        (len(context.intersection(ss.definition().split())), ss) for ss in synsets
    )

    return sense

In [45]:
#Function to extract adjectives from the contents. Return a list of strings
#Need nltk library: import nltk
def extract_adjective(sentence):
    adj_sentences = list()
    count = 0
    words = nltk.word_tokenize(sentence)
    adj_tags = nltk.pos_tag(words)
    one_adj_sentence = ""
    for index, tag in enumerate(adj_tags, start = 0):
        one_tag = tag[1]
        if one_tag in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VBP', 'VB', 'VBD', 'VBG','VBN']:
            count = count+1
    adj_sentences.append(one_adj_sentence)
    print(one_adj_sentence)
    return count

In [26]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [27]:
def encode_words(sentences):
    encoded = list()
    for sentence in sentences:
        words = list()
        mapping = list()
        words = nltk.word_tokenize(sentence)
        for word in words:
            mapping.append(one_hot(word,10000)[0])
        encoded.append(mapping);
    return encoded

In [28]:
def append_with_max_len(encoded,length):
    for e in encoded:
        for count in range(len(e), length):
            e.append(0)
    return encoded;

In [29]:
def set_label(fake_size, real_size):
    label = list()
    for counter in range(0,fake_size):
        label.append(0)
    for counter in range(0,real_size):
        label.append(1)
    return label

In [46]:
train_news = list()
fake_size = 0
real_size = 0
words = list()
with open('./data/titles/fake_news_training.txt') as train1:
    with open('./data/titles/real_news_training.txt') as train2:
        for line in train1:
            words.append(extract_adjective(clean_sentence(line))) 
            fake_size = fake_size+1
        for line in train2:
            words.append(extract_adjective(clean_sentence(line))) 
            real_size = real_size+1
predict_news = list()
count = 0
count1 = 0
with open('./data/real2.txt') as predict1:
    with open('./data/fake2.txt') as predict2:
        for line in predict1:
            predict_news.append(clean_sentence(line))
            count = count+1
        for line in predict2:
            predict_news.append(clean_sentence(line))
            count1 = count1+1
   
print((words))


1
